-----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Librerias

In [66]:
import pandas as pd
import json
from collections import defaultdict
import re
import numpy as np
from utils.utilities import Tiempo
import utils.dataExtraction
import statistics
import keyword
import re
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que extrae los timestamps de inicio y fin de cada intentos de nivel
#### También extrae las estrellas obtenidas en cada intentos y cuantas veces se han intentado todos los niveles por cada jugador 

In [55]:
def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1
                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que resta los timestamps de inicio y fin de cada intento de nivel para sacar la diferencia de tiempo

In [56]:
def tiempoPorNiveles_Jugador(data):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    timeDifference = Tiempo(times["ini"], times["fin"])
                    if level in tiemposJugados[player]:
                        tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                    else:
                        tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
    return tiemposJugados

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que devuelve el tiempo medio necesario para completar un nivel y un diccionario con el tiempo empleado por cada jugador hasta completar el nivel
(El tiempo necesario para cada jugador es la suma de todos los tiempos hasta conseguir completar el nivel)
* También devuelve la media de estrellas y una lista de los niveles ordenados

In [57]:
#TAMBIEN DEVUELVE EL TIEMPO INDIVIDUAL DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getMediaTiempoPorNivel(tiempos, soloPrimerExito = True, tiemposOrdenados = False):
    medias = defaultdict(list)
    mediasEstrellas = defaultdict(list)
    tiempoCompletarNivelIndividual = defaultdict(defaultdict)
    
    for player in tiempos:
        for level in tiempos[player]:
            tAux = Tiempo("0s")
            for t in tiempos[player][level]:
                if t["stars"] != -1:
                    if level in medias:
                        medias[level].append(int(tAux + t["time"]))
                        mediasEstrellas[level].append(int(t["stars"]))
                    else:
                        medias[level] = [int(tAux + t["time"])]
                        mediasEstrellas[level] = [int(t["stars"])]
                        
                    tiempoCompletarNivelIndividual[player][level] = tAux + t["time"]
                    if soloPrimerExito:
                        break
                elif t["stars"] == 0:
                    print("ALERTA")
                else:
                    tAux += t["time"]
    for m in medias:
        medias[m] = Tiempo(str(int(round(statistics.mean(medias[m]), 0))) + "s")
        
    for m in mediasEstrellas:
        mediasEstrellas[m] = statistics.mean(mediasEstrellas[m])
    
    listaNiveles = medias.keys()
    
    if tiemposOrdenados:
        medias = sorted(medias.items(), key=lambda x: x[1])
        mediasEstrellas = sorted(mediasEstrellas.items(), key=lambda x: x[1])
    else:
        medias = list(medias.items())
        mediasEstrellas = list(mediasEstrellas.items())
    
    return {"mediaTiempos" : medias, "mediaEstrellas" : mediasEstrellas, "listaNiveles" : listaNiveles, "tiemposIndividuales" : tiempoCompletarNivelIndividual}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve un diccionario con clave: id Jugador y valor: ultimo nivel alcanzado

In [58]:
def getUltimoNivelAlcanzado(tiempos):
    ultNivel = defaultdict()
    for player in tiempos:
        for level in tiempos[player]:
            ultNivel[player] = level
    return ultNivel

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve los intentos necesarios de cada jugador para completar un nivel al igual que los intentos medios

In [59]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

In [60]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [61]:
JSONFile = open('./data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

In [62]:
tiemposIntentosJugadores = tiempoPorNiveles_Jugador(resultados_Tiempos_Nivel_Jugador["tiempos"])
#pd.DataFrame(tiemposIntentosJugadores)

In [63]:
soloPrimerExito = True
tiemposOrdenados = False
tiemposMedios = getMediaTiempoPorNivel(tiemposIntentosJugadores, soloPrimerExito, tiemposOrdenados)
#pd.DataFrame(tiemposMedios["mediaTiempos"])
#pd.DataFrame(tiemposMedios["mediaEstrellas"])
#pd.DataFrame(tiemposMedios["listaNiveles"])
#pd.DataFrame(tiemposMedios["tiemposIndividuales"])

In [64]:
ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposMedios["tiemposIndividuales"])
#print(ultNivelAlcanzado)

In [65]:
intentosOrdenados = False
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)
#pd.DataFrame(intentosMedios_Individual["intentosMedios"])
#pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

In [76]:
def getGraficosIntentosCategoria(levels):
    levels_by_category = {}
    figures = []

    for level in levels:
        level_name, level_value = level
        match = re.search(r'(.*)_(\d+)', level_name)
        if match:
            level_category, level_num = match.group(1), int(match.group(2))
            if level_category not in levels_by_category:
                levels_by_category[level_category] = []
            levels_by_category[level_category].append((level_num, level_value))

    for category, category_levels in levels_by_category.items():
        category_levels.sort()
        values = [v for n, v in category_levels]
        levels_names = [f"{category}_{n}" for n, v in category_levels]
        fig, ax = plt.subplots()
        ax.bar(levels_names, values)
        ax.set_title(f"Niveles de {category}")
        ax.set_xlabel("Niveles")
        ax.set_ylabel("Intentos medios")
        plt.xticks(rotation=45)
        figures.append(fig)
    return figures

In [77]:
def getGraficosTiemposMedios(levels):
    levels_by_category = {}

    for level in levels:
        level_name, level_value = level
        match = re.search(r'(.*)_(\d+)', level_name)
        if match:
            level_category, level_num = match.group(1), int(match.group(2))
            if level_category not in levels_by_category:
                levels_by_category[level_category] = []
            levels_by_category[level_category].append((level_num, int(level_value)))


    for category, category_levels in levels_by_category.items():
        category_levels.sort()
        values = [v for n, v in category_levels]
        levels_names = [f"{category}_{n}" for n, v in category_levels]
        fig, ax = plt.subplots()
        ax.bar(levels_names, values)
        ax.set_title(f"Niveles de {category}")
        ax.set_xlabel("Niveles")
        ax.set_ylabel("Tiempos medios (s)")
        plt.xticks(rotation=45)


In [80]:
def getCorrelacionTiempoIntentos(tiempos, intentos):
    tiempos_dict = {nombre: int(tiempo) for nombre, tiempo in tiempos}
    intentos_dict = {nombre: intento for nombre, intento in intentos}

    # Crear arrays para calcular correlación
    x = []
    y = []
    for nombre in tiempos_dict.keys():
        if nombre in intentos_dict:
            x.append(tiempos_dict[nombre])
            y.append(intentos_dict[nombre])

    # Calcular correlación
    return np.corrcoef(x, y)[0, 1]

In [ ]:
graficosIntentos = getGraficosIntentosCategoria(intentosMedios_Individual["intentosMedios"])

In [ ]:
graficosTiempos = getGraficosTiemposMedios(tiemposMedios["mediaTiempos"])

In [81]:
getCorrelacionTiempoIntentos(tiemposMedios["mediaTiempos"], intentosMedios_Individual["intentosMedios"])

0.7112781935380275